In [2]:
import json
with open('/workspace/CS762_Project/Data_files/final_seed_data.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
args = {'local_rank': -1, 'save_total_limit': 5, 'per_device_train_batch_size': 4, 'per_device_eval_batch_size': 1, 'gradient_accumulation_steps': 4, 'learning_rate': 0.0002, 'max_grad_norm': 0.3, 'weight_decay': 0.001, 'lora_alpha': 16, 'lora_dropout': 0.1, 'lora_r': 64, 'max_seq_length': 512, 'model_name': '/workspace/CS762_Project/CodeLlama-7b-Python-hf', 'dataset_name': '/workspace/CS762_Project/generated_data', 'use_4bit': True, 'use_nested_quant': False, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_type': 'nf4', 'num_train_epochs': 1, 'fp16': True, 'bf16': False, 'packing': False, 'gradient_checkpointing': True, 'optim': 'paged_adamw_32bit', 'lr_scheduler_type': 'constant', 'max_steps': 10000, 'warmup_ratio': 0.03, 'group_by_length': True, 'save_steps': 10, 'logging_steps': 10, 'merge_and_push': False, 'output_dir': './results'}

In [13]:
import random
with open('/workspace/CS762_Project/Data_files/final_seed_data.json', 'r') as json_file:
        data = json.load(json_file)
for idx in range(len(data)):
    data[idx]['cluster'] = random.randint(0,4)
clustered_data = {}
for i in data:
    if i['cluster'] not in clustered_data:
        clustered_data[i['cluster']] = []
    clustered_data[i['cluster']].append(i)

In [14]:
{k:len(v) for k,v in clustered_data.items()}

{0: 16, 4: 17, 1: 8, 2: 9, 3: 8}

In [2]:
import os
from dataclasses import dataclass, field, asdict
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from datasets import DatasetDict, Dataset
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
@dataclass
class ScriptArguments:
    """
    These arguments vary depending on how many GPUs you have, what their capacity and features are, and what size model you want to train.
    """

    local_rank: Optional[int] = field(default=-1, metadata={"help": "Used for multi-gpu"})
    save_total_limit: Optional[int] = field(default=5)
    per_device_train_batch_size: Optional[int] = field(default=4)
    per_device_eval_batch_size: Optional[int] = field(default=1)
    gradient_accumulation_steps: Optional[int] = field(default=4)
    learning_rate: Optional[float] = field(default=2e-4)
    max_grad_norm: Optional[float] = field(default=0.3)
    weight_decay: Optional[int] = field(default=0.001)
    lora_alpha: Optional[int] = field(default=16)
    lora_dropout: Optional[float] = field(default=0.1)
    lora_r: Optional[int] = field(default=64)
    max_seq_length: Optional[int] = field(default=512)
    # num_train_epochs: Optional[int] = field(default=1)
    model_name: Optional[str] = field(
        default="/workspace/CS762_Project/CodeLlama-7b-Python-hf",
        metadata={
            "help": "The model that you want to train from the Hugging Face hub. E.g. gpt2, gpt2-xl, bert, etc."
        }
    )
    
    dataset_name: Optional[str] = field(
        default="/workspace/CS762_Project/generated_data",
        metadata={"help": "The preference dataset to use."},
    )
    use_4bit: Optional[bool] = field(
        default=True,
        metadata={"help": "Activate 4bit precision base model loading"},
    )
    use_nested_quant: Optional[bool] = field(
        default=False,
        metadata={"help": "Activate nested quantization for 4bit base models"},
    )
    bnb_4bit_compute_dtype: Optional[str] = field(
        default="float16",
        metadata={"help": "Compute dtype for 4bit base models"},
    )
    bnb_4bit_quant_type: Optional[str] = field(
        default="nf4",
        metadata={"help": "Quantization type fp4 or nf4"},
    )
    num_train_epochs: Optional[int] = field(
        default=1,
        metadata={"help": "The number of training epochs for the reward model."},
    )
    fp16: Optional[bool] = field(
        default=True,
        metadata={"help": "Enables fp16 training."},
    )
    bf16: Optional[bool] = field(
        default=False,
        metadata={"help": "Enables bf16 training."},
    )
    packing: Optional[bool] = field(
        default=False,
        metadata={"help": "Use packing dataset creating."},
    )
    gradient_checkpointing: Optional[bool] = field(
        default=True,
        metadata={"help": "Enables gradient checkpointing."},
    )
    optim: Optional[str] = field(
        default="paged_adamw_32bit",
        metadata={"help": "The optimizer to use."},
    )
    lr_scheduler_type: str = field(
        default="constant",
        metadata={"help": "Learning rate schedule. Constant a bit better than cosine, and has advantage for analysis"},
    )
    max_steps: int = field(default=10000, metadata={"help": "How many optimizer update steps to take"})
    warmup_ratio: float = field(default=0.03, metadata={"help": "Fraction of steps to do a warmup for"})
    group_by_length: bool = field(
        default=True,
        metadata={
            "help": "Group sequences into batches with same length. Saves memory and speeds up training considerably."
        },
    )
    save_steps: int = field(default=10, metadata={"help": "Save checkpoint every X updates steps."})
    logging_steps: int = field(default=10, metadata={"help": "Log every X updates steps."})
    merge_and_push: Optional[bool] = field(
        default=False,
        metadata={"help": "Merge and push weights after training"},
    )
    output_dir: str = field(
        default="./results",
        metadata={"help": "The output directory where the model predictions and checkpoints will be written."},
    )

In [4]:
args = ScriptArguments(local_rank=-1, save_total_limit=5, per_device_train_batch_size=4, per_device_eval_batch_size=1, gradient_accumulation_steps=4, learning_rate=0.0002, max_grad_norm=0.3, weight_decay=0.001, lora_alpha=4, lora_dropout=0.1, lora_r=8, max_seq_length=128, model_name='/workspace/CS762_Project/CodeLlama-7b-Python-hf', dataset_name='/workspace/CS762_Project/generated_data', use_4bit=True, use_nested_quant=False, bnb_4bit_compute_dtype='float16', bnb_4bit_quant_type='nf4', num_train_epochs=1, fp16=True, bf16=False, packing=False, gradient_checkpointing=True, optim='paged_adamw_32bit', lr_scheduler_type='constant', max_steps=10000, warmup_ratio=0.03, group_by_length=True, save_steps=10, logging_steps=10, merge_and_push=False, output_dir='./results')

In [5]:
def create_and_prepare_model(script_args, model_name):
    compute_dtype = getattr(torch, args.bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=args.use_4bit,
        bnb_4bit_quant_type=args.bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=args.use_nested_quant,
    )

    if compute_dtype == torch.float16 and args.use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    # Load the entire model on the GPU 0
    # switch to `device_map = "auto"` for multi-GPU
    device_map = {"": 0}

    model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        quantization_config=bnb_config, 
        device_map=device_map, 
        use_auth_token=True,
        trust_remote_code=True
    )
    
    # check: https://github.com/huggingface/transformers/pull/24906
    model.config.pretraining_tp = 1 

    peft_config = LoraConfig(
        lora_alpha=script_args.lora_alpha,
        lora_dropout=script_args.lora_dropout,
        r=script_args.lora_r,
        bias="none",
        task_type="CAUSAL_LM", 
        target_modules = ['Wqkv','out_proj','fc1','fc2']
        # target_modules = ['Wqkv']
        # target_modules = ['gate_proj', 'down_proj', 'up_proj', 'q_proj', 'v_proj', 'k_proj', 'o_proj']
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer

def train_model(script_args, total, model_name):
    model_dir = model_name.split('/')[-1]
    for k in range(total):
        output_dir = f'/workspace/CS762_Project/Results/{model_dir}/total_clusters_{total}/k_{k}'
        dataset_name = f'/workspace/CS762_Project/Model/k_{total}/generated_data_k_{total}_cluster_{k}'
        training_arguments = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=script_args.per_device_train_batch_size,
            per_device_eval_batch_size=script_args.per_device_eval_batch_size,
            gradient_accumulation_steps=script_args.gradient_accumulation_steps,
            evaluation_strategy='steps',
            eval_steps=script_args.save_steps,
            optim=script_args.optim,
            save_steps=script_args.save_steps,
            logging_steps=script_args.logging_steps,
            learning_rate=script_args.learning_rate,
            fp16=script_args.fp16,
            bf16=script_args.bf16,
            max_grad_norm=script_args.max_grad_norm,
            # max_steps=script_args.max_steps,
            warmup_ratio=script_args.warmup_ratio,
            group_by_length=script_args.group_by_length,
            lr_scheduler_type=script_args.lr_scheduler_type,
            num_train_epochs = script_args.num_train_epochs,
            save_total_limit=script_args.save_total_limit,
            metric_for_best_model='eval_loss',
            # save_steps=script_args.save_steps,
            save_strategy='steps'
        )
        
        model, peft_config, tokenizer = create_and_prepare_model(script_args, model_name)
        model.config.use_cache = False
        # dataset = load_dataset(script_args.dataset_name, split="train")
        full_dataset = DatasetDict.load_from_disk(dataset_name)
        # Fix weird overflow issue with fp16 training
        tokenizer.padding_side = "right"
        
        trainer = SFTTrainer(
            model=model,
            train_dataset= full_dataset['train'],
            eval_dataset = full_dataset['val'],
            peft_config=peft_config,
            dataset_text_field="prompt",
            max_seq_length=script_args.max_seq_length,
            tokenizer=tokenizer,
            args=training_arguments,
            packing=script_args.packing
        )
        
        trainer.train()

In [7]:
model_list = ['/workspace/CS762_Project/phi-2']
k_list = [1, 5, 10]
for model_name in model_list:
    for total in k_list:
        train_model(args,total, model_name)
        

Your GPU supports bfloat16, you can accelerate training with the argument --bf16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Your GPU supports bfloat16, you can accelerate training with the argument --bf16


...

Step,Training Loss,Validation Loss


Your GPU supports bfloat16, you can accelerate training with the argument --bf16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
